In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import scipy

from linear_geodesic_optimization.mesh.rectangle import Mesh as RectangleMesh
from linear_geodesic_optimization.optimization.laplacian import Computer as Laplacian
from linear_geodesic_optimization.optimization.curvature import Computer as Curvature
from linear_geodesic_optimization.optimization.smooth_loss import Computer as SmoothLoss
from linear_geodesic_optimization.plot import get_mesh_plot, get_heat_map

In [ ]:
def get_phi(mesh, mu, sigma):
    return np.exp(-np.linalg.norm((mesh.get_coordinates()[:, :2] - mu) / sigma, axis=1)**2)

def get_perturbation(mesh: RectangleMesh, curvature: Curvature, phi, target_difference):
    topology = mesh.get_topology()
    p = mesh.get_coordinates()[:, :2]
    curvature.reverse()

    scaling_factor_1 = np.max(np.abs([
        sum([
            dif_kappa_i_j * (phi_j - phi_i)
            for w in v.vertices()
            for j in (w.index(),)
            for phi_j, dif_kappa_i_j in ((phi[j], dif_kappa_i[j]),)
        ])
        for v in topology.vertices()
        if not v.is_on_boundary()
        for i in (v.index(),)
        for phi_i, dif_kappa_i in ((phi[i], curvature.dif_kappa_1[i]),)
    ]))
    scaling_factor_2 = np.max(np.abs([
        sum([
            dif_kappa_i_j * (phi_j - phi_i)
            for w in v.vertices()
            for j in (w.index(),)
            for phi_j, dif_kappa_i_j in ((phi[j], dif_kappa_i[j]),)
        ])
        for v in topology.vertices()
        if not v.is_on_boundary()
        for i in (v.index(),)
        for phi_i, dif_kappa_i in ((phi[i], curvature.dif_kappa_2[i]),)
    ]))
    scaling_factor = max(scaling_factor_1, scaling_factor_2)

    if scaling_factor == 0.:
        epsilon = 1.
    else:
        epsilon = target_difference / scaling_factor

    return epsilon * phi

In [ ]:
rng = np.random.default_rng()
def generate_neighbor(mesh, curvature, target_difference):
    p = mesh.get_coordinates()[:, :2]
    z = mesh.get_parameters()
    amin = np.amin(p, axis = 0)
    amax = np.amax(p, axis = 0)
    a, b = rng.uniform(amin, amax, (2, 2))
    mu = a
    # sigma = np.linalg.norm(a - b)
    sigma = rng.uniform(0, np.linalg.norm(amax - amin))

    phi = get_phi(mesh, mu, sigma)
    perturbation = get_perturbation(mesh, curvature, phi, target_difference)

    return z + perturbation

In [ ]:
def compute_curvature_loss(curvature, target_curvatures):
    curvature.forward()
    loss = 0.
    for index, target_curvature in target_curvatures:
        # if curvature.kappa_1[index] < 0:
        #     return np.inf
        loss += (curvature.kappa_G[index] - target_curvature)**2
    return loss / len(target_curvatures)

def compute_spherical_curvature_loss(curvature, target_curvatures):
    curvature.forward()
    loss = 0.
    for index, target_curvature in target_curvatures:
        if curvature.kappa_1[index] < 0:
            return np.inf
        loss += (curvature.kappa_1[index] - np.abs(target_curvature)**0.5)**2 \
            + (curvature.kappa_2[index] - target_curvature / np.abs(target_curvature)**0.5)**2
    return loss / len(target_curvatures)

def compute_loss(curvature, smooth_loss, target_curvatures, lambda_smooth):
    smooth_loss.forward()
    return compute_curvature_loss(curvature, target_curvatures) \
        + lambda_smooth * smooth_loss.loss

def compute_spherical_loss(curvature, smooth_loss, target_curvatures, lambda_smooth):
    smooth_loss.forward()
    return compute_spherical_curvature_loss(curvature, target_curvatures) \
        + lambda_smooth * smooth_loss.loss

In [ ]:
width = height = 30
scale = 10.

mesh = RectangleMesh(width, height, scale)
p = mesh.get_coordinates()[:, :2]
z = np.zeros(p.shape[0])
mesh.set_parameters(z)

topology = mesh.get_topology()

laplacian = Laplacian(mesh)
curvature = Curvature(mesh, laplacian)
smooth_loss = SmoothLoss(mesh, laplacian, curvature)

In [ ]:
# target_curvatures = [
#     (mesh.nearest_vertex(np.array([-scale / 4., -scale / 4.])).index(), 1.),
#     (mesh.nearest_vertex(np.array([0., 0.])).index(), -2.),
#     (mesh.nearest_vertex(np.array([scale / 4., scale / 4.])).index(), 1.),
# ]
target_curvatures = [
    (mesh.nearest_vertex(np.array([-scale / 4., -scale / 4.])).index(), 0.8472570534765698),
    (mesh.nearest_vertex(np.array([-scale / 4., 0.0])).index(), -0.32826099032036815),
    (mesh.nearest_vertex(np.array([-scale / 4., 2.5])).index(), 0.8470348268387511),
    (mesh.nearest_vertex(np.array([0.0, -scale / 4.])).index(), -0.3282609903203699),
    (mesh.nearest_vertex(np.array([0.0, 0.0])).index(), 0.12997364021747337),
    (mesh.nearest_vertex(np.array([0.0, scale / 4.])).index(), -0.3276135436383303),
    (mesh.nearest_vertex(np.array([scale / 4., -scale / 4.])).index(), 0.8470348268387529),
    (mesh.nearest_vertex(np.array([scale / 4., 0.0])).index(), -0.3276135436383303),
    (mesh.nearest_vertex(np.array([scale / 4., scale / 4.])).index(), 0.8472570534765644)
]

max_iters = width * height * 2
lambda_smooth = 0.00001

In [ ]:
z = mesh.get_parameters()
loss = compute_loss(curvature, smooth_loss, target_curvatures, lambda_smooth)
z_best = np.copy(z)
loss_best = loss

improvements = 0
backsteps = 0

for iteration, temperature in enumerate(np.linspace(1., 0., max_iters, False)):
    mesh.set_parameters(z)

    if iteration % 10 == 0:
        smooth_loss.forward()
        print(f'Loss at iteration {iteration} is {loss:0.6f} ({smooth_loss.loss})')

    z_new = generate_neighbor(mesh, curvature, rng.normal(scale = 0.1))
    mesh.set_parameters(z_new)
    loss_new = compute_loss(curvature, smooth_loss, target_curvatures, lambda_smooth)
    if loss_new < loss or np.exp(-(loss_new - loss) / temperature * 100) > rng.random():
        if loss_new < loss:
            improvements += 1
        else:
            backsteps += 1

        z = z_new
        loss = loss_new

    if loss < loss_best:
        z_best = np.copy(z)
        loss_best = loss

print(f'Loss at iteration {max_iters} is {loss}')
print(f'Improvements: {improvements / max_iters}')
print(f'Backsteps: {backsteps / max_iters}')

In [ ]:
mesh.set_parameters(z_best)
curvature.forward()
smooth_loss.forward()
for index, target_curvature in target_curvatures:
    print(f'At i = {index}, target curvature is {target_curvature:0.6f}; actual curvature is {curvature.kappa_G[index]:0.6f} = ({curvature.kappa_1[index]:0.6f}) * ({curvature.kappa_2[index]:0.6f})')
print(f'Smooth loss is {smooth_loss.loss:0.6f}')
print(f'Total loss is {compute_loss(curvature, smooth_loss, target_curvatures, lambda_smooth)}')
get_mesh_plot(mesh, '')
plt.show()

In [ ]:
vertices = mesh.get_coordinates()
x = list(sorted(set(vertices[:,0])))
y = list(sorted(set(vertices[:,1])))
z = vertices[:,2]

curvature.forward()
kappa = np.array(curvature.kappa_G).reshape(width, height).T
kappa[0,:] = 0.
kappa[-1,:] = 0.
kappa[:,0] = 0.
kappa[:,-1] = 0.
np.clip(kappa, -3, 3, kappa)
get_heat_map(x, y, kappa, 'Curvature')
plt.show()

In [ ]:
def loss(z = None):
    if z is not None:
        mesh.set_parameters(z)
    curvature.forward()
    curvature_loss = sum(
        (curvature.kappa_G[index] - target_curvature)**2
        for index, target_curvature in target_curvatures
    ) / len(target_curvatures)
    smooth_loss.forward()
    return curvature_loss + lambda_smooth * smooth_loss.loss

def dif_loss(z):
    mesh.set_parameters(z)
    curvature.reverse()
    dif_curvature_loss = np.zeros(z.shape)
    for index, target_curvature in target_curvatures:
        kappa = curvature.kappa_G[index]
        for dif_index, dif_kappa in curvature.dif_kappa_G[index].items():
            dif_curvature_loss[dif_index] += 2 * (kappa - target_curvature) * dif_kappa
    dif_curvature_loss /= len(target_curvatures)
    smooth_loss.reverse()
    return dif_curvature_loss + lambda_smooth * smooth_loss.dif_loss

iteration = 0
def diagnostics(self, x = None, f = None, context = None):
    global iteration
    if iteration % 10 == 0:
        print(f'{iteration}: {loss():0.6f} ({smooth_loss.loss})')
    iteration += 1

In [ ]:
minimizer_kwargs = {
    'method': 'L-BFGS-B',
    'jac': dif_loss,
    'callback': diagnostics,
    'options': {'maxiter': max_iters},
}
x = vertices[:, 0]
y = vertices[:, 1]
z_0 = np.sqrt(400 - x**2 - y**2)
z = scipy.optimize.minimize(loss, z_0, **minimizer_kwargs).x

In [ ]:
mesh.set_parameters(z)
curvature.forward()
smooth_loss.forward()
for index, target_curvature in target_curvatures:
    print(f'At i = {index}, target curvature is {target_curvature:0.6f}; actual curvature is {curvature.kappa_G[index]:0.6f} = ({curvature.kappa_1[index]:0.6f}) * ({curvature.kappa_2[index]:0.6f})')
print(f'Smooth loss is {smooth_loss.loss:0.6f}')
print(f'Total loss is {compute_loss(curvature, smooth_loss, target_curvatures, lambda_smooth)}')
get_mesh_plot(mesh, '')
plt.show()

In [ ]:
vertices = mesh.get_coordinates()
x = list(sorted(set(vertices[:,0])))
y = list(sorted(set(vertices[:,1])))
z = vertices[:,2]

curvature.forward()
kappa = np.array(curvature.kappa_G).reshape(width, height).T
kappa[0,:] = 0.
kappa[-1,:] = 0.
kappa[:,0] = 0.
kappa[:,-1] = 0.
np.clip(kappa, -3, 3, kappa)
get_heat_map(x, y, kappa, 'Curvature')
plt.show()